In [1]:
##indexing##
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
loader=WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        ),
    )
)

docs=loader.load()

#split
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300,chunk_overlap=50)
splits=text_splitter.split_documents(docs) #->list

#embed
embedding_model=HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")
vectorstore=Chroma.from_documents(documents=splits,embedding=embedding_model)
retriever=vectorstore.as_retriever()

#ollama llm
llm=ChatOllama(model="llama3.2:3b-instruct-q8_0",temperature=0)

c:\Users\user\anaconda3\envs\torch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Decomposition
template="""
You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):
"""

Decomposition_prompt=ChatPromptTemplate.from_template(template)

generate_queries_decomposition= (
    Decomposition_prompt
    | llm
    | StrOutputParser()
    | (lambda x : x.split("\n"))
)
question = "What is task decomposition for LLM agents?"


check_Decomposition=generate_queries_decomposition.invoke({"question":question})
check_Decomposition

['Here are three potential search queries related to "task decomposition for LLM agents":',
 '',
 '1. "What is the purpose of task decomposition in large language model (LLM) training?"',
 '2. "How does task decomposition affect the performance of LLM agents on specific tasks, such as question answering or text classification?"',
 '3. "Can you explain the different types of task decomposition methods used for LLMs, such as hierarchical task decomposition and subtask decomposition?"',
 '',
 'These queries can be answered in isolation to provide a comprehensive understanding of task decomposition for LLM agents.']

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langsmith import Client

client = Client()
prompt_rag = client.pull_prompt("rlm/rag-prompt")

#Answer individually
def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    sub_questions =sub_question_generator_chain.invoke({"question":question})

    rag_result=[]

    for sub_question in sub_questions:
        retrieved_docs=retriever.invoke(sub_question)

        answer = (
             prompt_rag
            | llm
            | StrOutputParser()
            ).invoke({"context":retrieved_docs, "question":sub_question})
        rag_result.append(answer)

    return rag_result,sub_questions

answer,questions=retrieve_and_rag(question,prompt_rag,generate_queries_decomposition)

In [8]:
def format_QA(answers,questions):
    format_string=""
    for i ,(question,answer) in enumerate (zip(questions,answers) , start=1):
        format_string=format_string+f"Question {i}: {question} \n answer{i}:{answer} \n\n"
    return format_string.strip()

context=format_QA(answer,questions)

template="""
Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt=ChatPromptTemplate.from_template(template)

final_rag_chain=(
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'Based on the provided Q&A pairs, task decomposition for LLM agents refers to the process of breaking down complex tasks into smaller, more manageable subgoals. This technique enables efficient handling of large tasks by allowing LLMs to think step-by-step and utilize more test-time computation.\n\nTask decomposition can be achieved through various methods, including:\n\n1. Hierarchical task decomposition: Breaking down complex tasks into smaller, more manageable subtasks, as seen in Chain of Thought (CoT) and Tree of Thoughts (Yao et al.). This method involves decomposing big tasks into multiple simpler steps.\n2. Subtask decomposition: Identifying specific subgoals within a larger task using simple prompting or task-specific instructions.\n\nThe benefits of task decomposition for LLM agents include:\n\n* Efficient handling of complex tasks\n* Improved planning, reflection, and memory capabilities\n\nTask decomposition can be done through various techniques, including:\n\n* Simple pro